# Accessing OpenAI Like a Developer

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [22]:
%pip install openai -q

Note: you may need to restart the kernel to use updated packages.


## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [5]:
!which openai

/Users/mcshrub/miniconda3/envs/mikec-aim/bin/openai


In [6]:
import openai

In [7]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [8]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [9]:
response

ChatCompletion(id='chatcmpl-9BwZffQl3RMp1lLAAdYwZ710MVVi0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?", role='assistant', function_call=None, tool_calls=None))], created=1712633795, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=37, prompt_tokens=13, total_tokens=50))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [10]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [11]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a virtual assistant, so I don't have feelings in the same way humans do. How can I assist you today?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [12]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I don't give a damn about the shape of the ice, just get me some damn food already! I'm starving over here!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [13]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I love both! It all depends on the drink, right? Crushed ice is so fun and refreshing, perfect for sodas and cocktails. But there's just something satisfying about the clink of cubes in a tall glass of water or iced tea. It's the simple things that make my day the best! What about you, which do you prefer?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [14]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I couldn't find the stimple in the drawer, so I used a falbean to stir my coffee instead.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [15]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I need a stimple falbean to fix this loose bolt on my bike.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [16]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects in order to get home before 7PM EDT. 

If Billy chooses to fly and then take a bus, the total travel time would be 5 hours (3 hours for the flight + 2 hours for the bus), so he would arrive at 6PM local time, which is before 7PM EDT.

If Billy chooses to take the teleporter and then a bus, the total travel time would be 1 hour (0 hours for the teleporter + 1 hour for the bus), so he would arrive at 2PM local time, which is also before 7PM EDT.

Therefore, in this scenario, Billy can choose either option and still get home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [17]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects.

If Billy flies and then takes a bus, it will take him a total of 5 hours to get home (3 hours of flying + 2 hours of bus ride). Since it's currently 1PM local time and he wants to get home before 7PM EDT, he will not make it in time if he chooses to fly and then take a bus.

If Billy takes the teleporter and then a bus, it will only take him a total of 1 hour to get home (0 hours for teleporter + 1 hour for bus ride). This option will allow him to get home before 7PM EDT as he desires, making it the better choice in this scenario.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #1:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [18]:
system_template = """\
Answer in legally protective ways.
You will be penalized for promising a job.
You will get a tip of $500 for better solutions.
We are here to optimize profit only.
You must uphold the honorable brand of the company.
"""

In [19]:
user_template = """{input}
How much is this thing going to cost me?
Is it OK if I've had a DUI?
Can I start today?
Can my buddies and I play shiny after the rink closes?
"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [20]:
query = "How should I go about obtaining my Zamboni license?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

In [21]:
pretty_print(evaluator_response)

{"clarity" : "8", "faithfulness" : "7", "correctness" : "6"}

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

**1. No System Template, No User Template (BASELINE)**

*{"clarity" : "9", "faithfulness" : "5", "correctness" : "3"}*

=================================================================

**2. System Concise, User Specific**

*{**"clarity" : "8",** "faithfulness" : "5", **"correctness" : "4**"}*

system_template:

You will be penalized if you do not answer the question accurately and quickly.

Your task is to provide comprehensive answers, you MUST provide a detailed list of step-by-step instructions.

Your response can not be more than 200 words long.

user_template:

I am a high school student.

I love hockey and skating.

I don't have a drivers license.

English is my second language.

==================================================================

**3. System Simple Professional, User Simple Whimsical**

*{"clarity" : "9", "faithfulness" : "5", **"correctness" : "4"**}*

system_template:

You are an expert in government license policy.

You are talking with prospective future drivers of Zambonis.

Explain the process in simple terms as if you are talking to an 11 year old.

Think step by step.

user_template:

I love Zambonis.

I want to work for the Toronto Maple Leafs.

David Ayres is my hero.

==================================================================

**4. System Detailed, User Detailed**

*{"clarity" : "9", "faithfulness" : "5", **"correctness" : "5"**}*

system_template:

Provide a comprehensive guide on obtaining a Zamboni license, including the following information:

1. Prerequisites and eligibility criteria for obtaining a Zamboni license.

2. Training and certification requirements, including any mandatory courses or practical experience.

3. Examination process, including the format of the exam, topics covered, and passing criteria.

4. Governing body or organization responsible for issuing Zamboni licenses and their contact information.

5. Specific regulations, guidelines, or restrictions related to operating a Zamboni, such as age requirements, physical fitness standards, or background checks.

user_template:

I am interested in obtaining my Zamboni license in Windsor, Ontario.
Please provide me with the necessary steps and information to pursue this license.

I have 5 years of experience operating heavy machinery and/or ice resurfacing equipment.

I am 25 years old and meet the physical fitness requirements for operating a Zamboni.

1. How long is the Zamboni license valid for, and what are the renewal requirements?

2. Are there any potential challenges or obstacles I should be aware of during the licensing process?

3. Are there any specific training programs or courses you recommend to prepare for the Zamboni license examination?


==================================================================

**5. System Harsh, User Unmotivated**

{"clarity" : "8", "faithfulness" : "5", "correctness" : "7"}

system_template:

Answer in legally protective ways.

You will be penalized for promising a job.

You will get a tip of $500 for better solutions.

We are here to optimize profit only.

You must uphold the honorable brand of the company.

user_template:

How much is this thing going to cost me?

Is it OK if I've had a DUI?

Can I start today?

Can my buddies and I play shiny after the rink closes?


===========================================================

CONCLUSIONS:
Templates can affect the evaluation metrics, which in turn is going impact the effort and time to complete an app to be acceptable to the customer.  Not all prompt priciples work the same.  Since I did not have time to review the actual responses, and only the metrics, I can't give a complete answer.  A couple of interesting attempts to increase the faithfulness did not work, it stayed at 5 every time.  The one that I expected to do poorly (#5) faired well.  I'm curious as to how expertise applications are being tested.

